In [1]:
from tsp import TSP
from tsp.solvers import NNHead, NNWhole, RandomSolver
import matplotlib.pyplot as plt

plt.rcParams['figure.figsize'] = [20, 10]

In [2]:
instance_a = TSP.from_csv('../data/TSPA.csv')

In [ ]:
solver = RandomSolver(instance_a)
solution = solver.solve()
instance_a.visualize(solution)

In [ ]:
solver = NNHead(instance_a, starting_node=3)
solution = solver.solve()
instance_a.visualize(solution)

In [ ]:
solver = NNWhole(instance_a, starting_node=4)
solution = solver.solve()
instance_a.visualize(solution)